# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict
import pickle
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger', 'tagsets'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [4]:
# load data from database
engine = create_engine('sqlite:///disaster_messages_v3.db')
df = pd.read_sql_table('disastermessages_v3', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [3]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    '''
    Returns the tokenized form of the input text.

            Parameters:
                    text (str): The input text to be tokenized

            Returns:
                    tokenized (list): The list of tokens extracted from text
    '''
    
    # normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize
    tokens = word_tokenize(text)
    
    # stemming and stop-words removal
    stemmer = PorterStemmer()
    stop_words = stopwords.words('english')
    
    tokenized = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    return tokenized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer = tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(19303,)
(19303, 36)
(6435,)
(6435, 36)


In [8]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [9]:
train_preds_v1 = pipeline.predict(X_train)

In [10]:
test_preds_v1 = pipeline.predict(X_test)

In [11]:
print(classification_report(Y_test.loc[:, 'related'], test_preds_v1[:, 0]))

             precision    recall  f1-score   support

          0       0.62      0.51      0.56      1457
          1       0.86      0.91      0.89      4978

avg / total       0.81      0.82      0.81      6435



### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def report_metrics(truths, predicted):
    '''
    Returns the .

            Parameters:
                    truths (np.array): The y_train, the truth labels for X_train
                    predicted (np.array): The prediction of the model

            Returns:
                    df (dataframe): A dataframe containing the various metrics
                                    for each category type
    '''
    
    col_names = list(truths.columns)
    metrics = list()
    
    for i, column in enumerate(col_names):
        try:
            # for binary columns
            accuracy = accuracy_score(truths.loc[:, column], predicted[:, i])
            f1 = f1_score(truths.loc[:, column], predicted[:, i])
            precision = precision_score(truths.loc[:, column], predicted[:, i])
            recall = recall_score(truths.loc[:, column], predicted[:, i])
        except:
            # for multi label columns (which is just the 'related' column for now)
            accuracy = accuracy_score(truths.loc[:, column], predicted[:, i])
            f1 = f1_score(truths.loc[:, column], predicted[:, i], average='weighted')
            precision = precision_score(truths.loc[:, column], predicted[:, i], average='weighted')
            recall = recall_score(truths.loc[:, column], predicted[:, i], average='weighted')
        
        metrics.append([accuracy, f1, precision, recall])
        
    df = pd.DataFrame(metrics, index=col_names, columns=['accuracy', 'f1_score', 'precision', 'recall'])
    return df
    

In [13]:
# report on train
report_metrics(Y_train, train_preds_v1)

,accuracy,f1_score,precision,recall
related,0.989483,0.993116,0.991670,0.994566
request,0.987256,0.961272,0.998039,0.927118
offer,0.998601,0.830189,1.000000,0.709677
aid_related,0.984458,0.981120,0.994387,0.968203
medical_help,0.987878,0.919031,1.000000,0.850192
medical_products,0.991867,0.913784,0.998800,0.842105
search_and_rescue,0.994819,0.894068,1.000000,0.808429
security,0.994923,0.839344,1.000000,0.723164
military,0.995493,0.927440,0.998205,0.866044
child_alone,1.000000,0.000000,0.000000,0.000000


In [14]:
# report on test
report_metrics(Y_test, test_preds_v1)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy,f1_score,precision,recall
related,0.819425,0.886413,0.863290,0.910808
request,0.880031,0.544274,0.780034,0.417951
offer,0.996115,0.000000,0.000000,0.000000
aid_related,0.752758,0.676033,0.757299,0.610519
medical_help,0.924631,0.195688,0.621053,0.116142
medical_products,0.952137,0.109827,0.655172,0.059937
search_and_rescue,0.969852,0.049020,0.555556,0.025641
security,0.981974,0.016949,0.333333,0.008696
military,0.967832,0.133891,0.761905,0.073394
child_alone,1.000000,0.000000,0.000000,0.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
pipeline.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fddc8550620>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,

In [16]:
parameters = {
    'tfidf__ngram_range' : ((1,1), (1,2), (1,3), (1,4)),
    'tfidf__norm': ('l1', 'l2', None),
}

cv = GridSearchCV(pipeline, param_grid=parameters,scoring='f1_weighted', n_jobs=-1, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [34]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] tfidf__ngram_range=(1, 1), tfidf__norm=l1 .......................


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV] ........ tfidf__ngram_range=(1, 1), tfidf__norm=l1, total=  58.0s
[CV] tfidf__ngram_range=(1, 1), tfidf__norm=l1 .......................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV] ........ tfidf__ngram_range=(1, 1), tfidf__norm=l1, total=  57.1s
[CV] tfidf__ngram_range=(1, 1), tfidf__norm=l1 .......................


...

[CV] ...... tfidf__ngram_range=(1, 4), tfidf__norm=None, total= 4.8min
[CV] tfidf__ngram_range=(1, 4), tfidf__norm=None .....................


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV] ...... tfidf__ngram_range=(1, 4), tfidf__norm=None, total= 4.9min


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 112.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__ngram_range': ((1, 1), (1, 2), (1, 3), (1, 4)), 'tfidf__norm': ('l1', 'l2', None)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=2)

In [35]:
test_preds_v2 = cv.predict(X_testst)
report_metrics(Y_test, test_preds_v2)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy,f1_score,precision,recall
related,0.811033,0.879221,0.869548,0.889112
request,0.882673,0.573205,0.761261,0.459655
offer,0.996270,0.000000,0.000000,0.000000
aid_related,0.752914,0.675112,0.759540,0.607576
medical_help,0.920124,0.157377,0.470588,0.094488
medical_products,0.953069,0.165746,0.666667,0.094637
search_and_rescue,0.971251,0.155251,0.708333,0.087179
security,0.981507,0.016529,0.166667,0.008696
military,0.967211,0.166008,0.600000,0.096330
child_alone,1.000000,0.000000,0.000000,0.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [7]:
class MetaData(BaseEstimator, TransformerMixin):
    
    def count_meta_data(self, text):
        counter = defaultdict(int)
        
        # tokenize by sentences
        sentence_list = sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(word_tokenize(sentence))
            
            # check each tags word and keep a count of verbs
            for _, tag in pos_tags:
                if tag.startswith('JJ'):
                    counter['adjective'] += 1
                elif tag.startswith('NN'):
                    counter['noun'] += 1
                elif tag.startswith('PRP'):
                    counter['pronoun'] += 1
                elif tag.startswith('RB'):
                    counter['adverb'] += 1
                elif tag.startswith('VB'):
                    counter['verb'] += 1
            
            return counter

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply count meta data for each text
        X_tagged = pd.Series(X).apply(lambda x : self.count_meta_data(x)).values
        df = pd.DataFrame.from_records(X_tagged)
        df.fillna(0, inplace=True)
        df = df.astype(int)
        return df

In [8]:
pipelineV2 = Pipeline([
    ('features', FeatureUnion([
            ('tfidf', TfidfVectorizer(tokenizer = tokenize)),
            ('meta_data', MetaData())
        ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [9]:
parameters = {
    'features__tfidf__min_df' : (5, 10),
    'clf__estimator__n_estimators' : (30, 60, 100)
}

cv2 = GridSearchCV(pipelineV2, param_grid=parameters,scoring='f1_weighted', n_jobs=-1, verbose=2)

In [12]:
cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__n_estimators=30, features__tfidf__min_df=5 ......


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=30, features__tfidf__min_df=5, total= 4.2min
[CV] clf__estimator__n_estimators=30, features__tfidf__min_df=5 ......


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  5.5min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=30, features__tfidf__min_df=5, total= 2.8min
[CV] clf__estimator__n_estimators=30, features__tfidf__min_df=5 ......


...

[CV]  clf__estimator__n_estimators=100, features__tfidf__min_df=10, total= 4.7min
[CV] clf__estimator__n_estimators=100, features__tfidf__min_df=10 ....


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=100, features__tfidf__min_df=10, total= 4.7min


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 78.2min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_r...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'features__tfidf__min_df': (5, 10), 'clf__estimator__n_estimators': (30, 60, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=2)

In [13]:
test_preds_v3 = cv2.predict(X_test)
report_metrics(Y_test, test_preds_v3)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy,f1_score,precision,recall
related,0.821445,0.892204,0.837001,0.955203
request,0.894172,0.611079,0.825617,0.485041
offer,0.996270,0.000000,0.000000,0.000000
aid_related,0.782129,0.735072,0.755927,0.715337
medical_help,0.925563,0.223663,0.633028,0.135827
medical_products,0.954934,0.198895,0.800000,0.113565
search_and_rescue,0.970319,0.094787,0.625000,0.051282
security,0.981352,0.000000,0.000000,0.000000
military,0.968609,0.210938,0.710526,0.123853
child_alone,1.000000,0.000000,0.000000,0.000000


### 9. Export your model as a pickle file

In [14]:
final_model = cv2.best_estimator_
print(final_model)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


In [15]:
filename = 'finalized_model.pkl'
pickle.dump(final_model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.